In [25]:
import numpy as np
import os, sys, glob
import matplotlib.pyplot as plt
import open3d as o3d
from scipy.stats import chisquare
import pandas as pd

basedir = os.path.dirname(os.getcwd())
_py = os.path.join(basedir, 'py')
_data = os.path.join(basedir, 'data')
_images = os.path.join(basedir, 'images')

sys.path.insert(1, _py)
import lad
import figures

from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
# define the path of the data stored
mockname = 'single_tmp'
datapath = os.path.join(_data, mockname)

# read the numpy files
for file in glob.glob(os.path.join(datapath, 's*.numpy')):
    df = np.loadtxt(file)
    filename = file.split('/')[-1].split('.')[0]

    outfile = os.path.join(_data, mockname, filename)
    np.save(outfile, df)

# define the path of the data stored
mockname = 'single'
datapath = os.path.join(_data, mockname, 'toy_trees')
file = glob.glob(os.path.join(datapath, 'tree*.npy'))
print(file)
df = np.load(file[0])

In [123]:
mockname = 'single_tmp'
datapath = os.path.join(_data, mockname)

# read the numpy files
files = {}
plt.figure(figsize=(18,14))
for file in glob.glob(os.path.join(datapath, 's*.npy')):
    filename = file.split('/')[-1]
    df = np.load(file)
    # df = df[::1]
    yaw = df.T[1]
    pitch = df.T[2]
    x, y = df.T[5], df.T[6]

    
    plt.subplot(3,2,1)
    plt.title('yaw')
    #np.linspace(2.8, 3.5, 40)
    plt.hist(np.rad2deg(yaw), bins=40, histtype='step', lw=2, label=filename)
    plt.legend()
    plt.subplot(3,2,2)
    plt.title('pitch')
    #np.linspace(-0.35, 0.3, 40)
    plt.hist(np.rad2deg(pitch), bins=40, histtype='step', lw=2, label=filename)
    plt.subplot(3,2,3)
    plt.title('x')
    plt.hist(x, bins=40, histtype='step', lw=2, label=filename)
    plt.subplot(3,2,4)
    plt.title('y')
    plt.hist(y, bins=40, histtype='step', lw=2, label=filename)
    plt.subplot(3,2,5)
    uv = lad.beam_direction(yaw, pitch, scan_inc=90)
    beam_angs = []
    for i in uv:
        beam_angs.append(lad.vecttoangle([0, 0, 1], i))
    plt.hist(np.array(beam_angs), 40, histtype='step', label='pre correction')
    plt.legend()
    plt.subplot(3,2,6)
    plt.hist(uv.T[0], bins=40, histtype='step', lw=2, label=filename+' x')
    plt.hist(uv.T[1], bins=40, histtype='step', lw=2, label=filename+' y')
    plt.hist(uv.T[2], bins=40, histtype='step', lw=2, label=filename+' z')
    plt.legend()


In [130]:
colors = ['b', 'cyan', 'orange', 'yellow']
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection='3d')

for num, file in enumerate(glob.glob(os.path.join(datapath, 's1*.npy'))):
    
    filename = file.split('/')[-1]
    df = np.load(file)
    df = df[::10]
    print(df.shape)
    # yaw = df.T[1]
    # pitch = df.T[2]
    x, y, z = df.T[5], df.T[6], df.T[7]
    # sensor coordinates
    spos = os.path.join(datapath, 'scanner_pos.txt')
    scan = lad.laod_scan_pos(spos)
    id = [i.decode("utf-8") for i in scan['scan']]
    keep = np.array(id) == filename[:2]
    _, sx, sy, sz = scan[keep][0]
    print(scan[keep][0])

    ax.scatter3D(x, y, z, c='g', s=1)
    ax.scatter3D(sx, sy, sz, c='r', s=10)

    p2 = [sx, sy, sz]
    res = 2

    for i, j, k in zip(x, y, z):

        p1 = [i, j, k]

        px, py, pz = lad.line2points_vect(p1, p2, res=res)
        # ax.scatter3D(px, py, pz, c='k', s=1)
        ax.plot3D([i, sx], [j, sy], [k, sz], lw=0.1, color=colors[num])


(9000, 16)
(b's1', 10., 0., 15.)


In [19]:
# define the path of the data stored
mockname = 'single_tmp'
datapath = os.path.join(_data, mockname)

# read the numpy files
files = {}
bia_pos = {}
for file in glob.glob(os.path.join(datapath, 's*.npy')):
    df = np.load(file)
    filename = file.split('/')[-1]

    # sensor coordinates
    spos = os.path.join(datapath, 'scanner_pos.txt')
    scan = lad.laod_scan_pos(spos)
    id = [i.decode("utf-8") for i in scan['scan']]
    keep = np.array(id) == filename[:2]
    _, sx, sy, sz = scan[keep][0]

    if df.shape[0] > 0:
        files[filename] = df
        bia = np.full((len(df),3), np.array([sx, sy, sz]))
        bia_pos[filename] = bia
        print(filename, df.shape, bia.shape)
    else:
        print('file %s empty' %(filename))

    
    
# concatenate all data
df = np.concatenate(list(files.values()))
bias = np.concatenate(list(bia_pos.values()))
print(df.shape)
print(bias.shape)

s100000.npy (90000, 16) (90000, 3)
s300000.npy (90000, 16) (90000, 3)
s200000.npy (90000, 16) (90000, 3)
s400000.npy (90000, 16) (90000, 3)
(360000, 16)
(360000, 3)


In [20]:
# pass this to a pandas data frame for simplicity
scan = pd.DataFrame(df, columns=['timestamp', 'yaw', 'pitch', 'distance','distance_noise',
                                'x','y','z',
                                'x_noise','y_noise','z_noise',
                                'object_id', 'color0', 'color1','color2', 'idx'])

bia = pd.DataFrame(bias, columns=['sx', 'sy', 'sz'])

# concat
scan = pd.concat((scan, bia), axis=1)

In [21]:
scan

,timestamp,yaw,pitch,distance,distance_noise,x,y,z,x_noise,y_noise,z_noise,object_id,color0,color1,color2,idx,sx,sy,sz
0,0.0,2.792527,-0.349066,18.305555,18.303818,1.339747,-5.923964,-9.536743e-07,1.340569,-5.923401,0.001424,1.851878e+09,204.0,204.0,204.0,0.0,10.0,0.0,15.0
1,0.0,2.792527,-0.346922,18.329782,18.323534,1.296808,-5.935935,-1.430511e-06,1.299775,-5.933912,0.005113,1.851878e+09,204.0,204.0,204.0,1.0,10.0,0.0,15.0
2,0.0,2.792527,-0.344774,18.354183,18.358143,1.253694,-5.947956,-9.536743e-07,1.251808,-5.949239,-0.003236,1.851878e+09,204.0,204.0,204.0,2.0,10.0,0.0,15.0
3,0.0,2.792527,-0.342623,18.378759,18.382126,1.210407,-5.960025,9.536743e-07,1.208797,-5.961117,-0.002747,1.851878e+09,204.0,204.0,204.0,3.0,10.0,0.0,15.0
4,0.0,2.792527,-0.340469,18.403515,18.395956,1.166944,-5.972144,1.430511e-06,1.170571,-5.969691,0.006163,1.851878e+09,204.0,204.0,204.0,4.0,10.0,0.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359995,0.0,3.488514,0.338311,28.221243,28.224992,9.109637,15.000000,5.595051e+00,9.110847,15.003321,5.593802,1.851878e+09,204.0,204.0,204.0,89995.0,0.0,-10.0,15.0
359996,0.0,3.488514,0.340469,28.196182,28.200177,9.095338,14.999998,5.656572e+00,9.096626,15.003540,5.655249,1.851878e+09,204.0,204.0,204.0,89996.0,0.0,-10.0,15.0
359997,0.0,3.488514,0.342623,28.171324,28.164093,9.081086,15.000002,5.717903e+00,9.078755,14.993584,5.720286,1.851878e+09,204.0,204.0,204.0,89997.0,0.0,-10.0,15.0
359998,0.0,3.488514,0.344774,28.146656,28.147397,9.066877,14.999998,5.779041e+00,9.067116,15.000656,5.778797,1.851878e+09,204.0,204.0,204.0,89998.0,0.0,-10.0,15.0


In [22]:
### get the leaves only
leaves = scan['object_id'] == 1986094444.0
points = np.vstack((np.array(scan['x']), np.array(scan['y']), np.array(scan['z']))).transpose()
# pcd = lad.points2pcd(points)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[leaves])
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=np.asarray(pcd.points).shape))
o3d.visualization.draw_geometries([pcd])

In [23]:
# Here we show how some of the point cloud parameters looks like.
cols = ['x','y']

plt.figure(figsize=(18, 4*3))

for num, i in enumerate(cols):
    
    plt.subplot(3,3,num+1)
    plt.hist(scan[i][leaves], 80, alpha=0.5, label=i)
    plt.legend()
    
plt.show()

plt.figure(figsize=(14, 8))
plt.scatter(scan['x'][leaves], scan['y'][leaves])

In [24]:
#
plt.figure(figsize=(14, 8))

# x = np.linspace(0, 16, 3)
# y = np.array([0, 8])

# centres
x, y = [0], [0]

trees = {}
num = 0
# dx, dy = 4, 4
dx, dy = 5, 5

resdir_name = 'toy_trees'
resdir = os.path.join(datapath, resdir_name)
if not os.path.exists(resdir):
    os.makedirs(resdir)

for i in x:
    for j in y:
        
        keep = np.ones(len(scan['x']), dtype=bool)
        keep &= (scan['x'] < i+dx) & (scan['x'] > i-dx)
        keep &= (scan['y'] < j+dy) & (scan['y'] > j-dy)
        
        plt.scatter(scan['x'][leaves & keep], scan['y'][leaves & keep], s=0.5, label=num)
        
        np.save(os.path.join(resdir, 'tree_%s' %(str(num))), scan[leaves & keep])
        
        num += 1
        
plt.legend()   

In [136]:
# get true leag-angle-distribution
file = "../data/single_tmp/mesh.ply"
mesh = o3d.io.read_triangle_mesh(file)
# 
mesh.compute_triangle_normals()
mesh.compute_vertex_normals()
# print(mesh.cluster_connected_triangles())
# print("Computing normal and rendering it.")
# 
print(np.asarray(mesh.triangle_normals))
print(np.asarray(mesh.triangles))
o3d.visualization.draw_geometries([mesh], mesh_show_wireframe=True)


[[0.37349295 0.25693007 0.89134166]
 [0.3734951  0.25692643 0.89134181]
 [0.37349183 0.25692803 0.89134272]
 ...
 [0.87693035 0.28342595 0.38815318]
 [0.87693022 0.28342695 0.38815275]
 [0.87693417 0.28341742 0.38815077]]
[[    0     1     2]
 [    0     2     3]
 [    0     3     4]
 ...
 [41138 41140 41141]
 [41142 41143 41144]
 [41142 41144 41145]]


In [6]:
# get leaf area
file = "../data/spherical_02/mesh.ply"
mesh = o3d.io.read_triangle_mesh(file)
cidx, nt, area = mesh.cluster_connected_triangles()
la = np.array(area)[np.array(nt) == 4][0]
np.round(la, 4)

0.0267

In [17]:
len(area)

7144

In [11]:
set(np.round(np.array(area), 5))

{0.01333, 0.02667}

In [21]:
np.array(area)[np.array(nt) == 4][0]

0.026666766174740775